<a href="https://colab.research.google.com/github/fpaleixo/arbolom/blob/main/ARBoLoM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title ASP Revision of Boolean Logical Models

#Notebook by Frederico Aleixo

In [2]:
#@title ARBoLoM directory setup
%cd "/content/drive/MyDrive/FCT/5o ano/2o semestre/arbolom"

/content/drive/MyDrive/FCT/5o ano/2o semestre/arbolom


In [ ]:
#Usage: !python corruption.py -op (OPERATIONS) -(O)p (PROBABILITY)
#Variables: 
#OPERATIONS - A string with one (or more) specific characters, denoting which corruptions to apply. These characters are 'f','e','r' and 'a'. 'fera' would be the full string, representing that (f)unction change, (e)dge flip, edge (r)emove and edge (a)dd will all be applied.
#O - A character that can take one of four possible values: 'f','e','r' and 'a' (followed by 'p'). -fp would change the probability of function change to occur, -ep of edge removal, etc
#PROBABILITY - A float from 0.0 to 1.0 denoting the probability of a given corruption to occur. For example, -ap 0.5 would change the add edge operation's probability to 50%

!python corruption.py -op fra 